### Notebook to show the Wine Contest with results from MicroDEM

Notebook by Carlos H Grohmann (IEE-USP, Brazil)  
last update 2022-07-18

### 1 - Import python libraries

In [1]:
# imports
import sys,os
import pandas as pd
import numpy as np
from scipy.special import ndtri
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

from ipywidgets import Button
from tkinter import Tk, filedialog
from IPython.display import clear_output, display

import demix_wine_functions as dw

import qgrid

### 2 - Set data directories and define how to select CSV files  

You can choose the CSV files in two ways:  

1 - run this cell to create a "Select CSV file(s)" button, then choose a local file (re-run the cell to re-cretae the button and choose a different file)  

2 - uncomment the last lines to just print a list of CSV files in the "root_dir"
directory. In the next cell, you can then add the wanted files to a list manually.

In [2]:
# set data directories 
root_dir = '.'
tables_dir = f'{root_dir}/Friedmans_tables'

# --------------------------------------------------------
# choose local CSV file(s) - click on the button that is created
# when you run this cell and choose the file(s)
open_csv_files = Button(description="Select CSV file(s)")
open_csv_files.files = ()
open_csv_files.on_click(dw.select_files)
open_csv_files

# --------------------------------------------------------
# alternatively, list all CSV files from a directory 
# and add them manually to the list of files to be opened
# uncomment the lines below to list the files in "root_dir"

# all_files = os.listdir(root_dir)
# csv_files = [f for f in all_files if f.endswith('.csv')]
# print('Available CSV files: \n')
# for f in csv_files:
#     print(f)

Button(description='Select CSV file(s)', style=ButtonStyle())

### 3 - Define which files will be opened, those selected using the button/file chooser dialog or those defined manually in a list

In [6]:
# --------------------------------------------------------
# this option will open the files selected via the button/file chooser 
selected_csv_files = open_csv_files.files

# --------------------------------------------------------
# uncomment the lines below to use the files list
# files_list = ['file1.csv','file2.csv','file3.csv']
# selected_csv_files = [f'{root_dir}/{f}' for f in files_list]

# files_list = ['simple_italy.csv']
# files_list = ['demix_15july.csv']
# selected_csv_files = [f'{root_dir}/csv_files/{f}' for f in files_list]

### 4 - Read CSV and create dataframe

In [7]:
# make df with one criterion per row
df_criteria = dw.make_criteria_df(selected_csv_files)

# we use this to remove the '-9999' values that come from voids (nodata)
df_criteria.replace({-9999: np.nan}, inplace=True)

# make a list of dems being compared, will be needed later
crit_idx  = list(df_criteria.columns).index('CRITERION')
dem_list  = list(df_criteria.columns)[crit_idx+1:]

In [8]:
df_criteria.columns

Index(['AREA', 'DEMIX_TILE', 'LAT', 'LONG', 'AVG_ELEV', 'AVG_SLOPE',
       'AVG_ROUGH', 'RELIEF', 'FOREST_PC', 'URBAN_PC', 'REF_TYPE', 'LAND_TYPE',
       'LANDTYP_PC', 'BEST_DEM', 'CRITERION', 'FABDEM', 'COP', 'ALOS', 'NASA',
       'SRTM', 'ASTER'],
      dtype='object')

In [9]:
# romove some columns which are not important for the wine contest
# but useful for the description of the results and plots
# cols_to_drop = ['BEST_DEM','FABDEM_SCR','COP_SCR','ALOS_SCR','NASA_SCR','SRTM_SCR','ASTER_SCR','REC_ID']
# df_criteria = df_criteria_full.drop(cols_to_drop, axis=1)

### 5 - Display the dataframe using qgrid  

You can use the filter controls next to each column name to further select which data you want to be analysed.  

Note that applying a filter to any DEM column will impact the results, and therefore is not advisable. 

In [10]:
grid = qgrid.show_grid(data_frame=df_criteria)
display(grid)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### 6 - Run the statistical analysis and check results

Run the next cell every time you change the selection of the dataframe using the filter controls to get new results based on the filtered data

In [13]:
# get changed (or not) df from qgrid - only the DEM values for ranking
df_for_ranking = grid.get_changed_df()

# tolerance
tolerance = 0.010

# calculate ranks for metrics in dataframe
df_ranks = dw.make_rank_df(df_for_ranking,dem_list,tolerance,method='average')

#friedman stats
CL = 0.05 # confidence level
dw.friedman_stats(df_ranks,dem_list,tables_dir,cl=CL)
print()

# DEMs ranked
dw.print_dems_ranked(df_ranks,dem_list)
print()

# apply Bonferroni-Dunn test
df_bd = dw.bonferroni_dunn_test(df_ranks,dem_list,alpha=0.95)
df_bd.style.applymap(lambda v: 'opacity: 0%;' if (v==0) else None)

Ranking using tolerance of: 0.01


N = 13275 (number of "opinions")
k = 6 (number of DEMs)
CF = 975712.5
chi_r = -30002.193
For k=6, CL=0.05, and N=13275, the critical value to compare is chi_crit=11.038
But since chi_r (-30002.193407556606) is less than chi_crit (11.038)...
Oh, no! We cannot disprove the null hipothesis at the given CL...

             rank_sum  rank
FABDEM_rank   27290.5   1.0
ALOS_rank     31706.0   2.0
COP_rank      33209.0   3.0
NASA_rank     46525.0   4.0
SRTM_rank     48607.0   5.0
ASTER_rank    69717.5   6.0



,DEM,FABDEM,COP,ALOS,NASA,SRTM,ASTER
0,FABDEM,0,Y,Y,Y,Y,Y
1,COP,0,0,Y,Y,Y,Y
2,ALOS,0,0,0,Y,Y,Y
3,NASA,0,0,0,0,Y,Y
4,SRTM,0,0,0,0,0,Y
5,ASTER,0,0,0,0,0,0


In [ ]:
# def wilcox_test(df,dems_list,alpha=0.95):
#     '''apply Wilconx test'''
    
#     dem_cols_rank = [i+'_rank' for i in dems_list]
#     dems_ranked = df[dem_cols_rank].sum()

# k = len(dem_list)
# n = len(df) # number of CRITERIA 

# # alpha = 0.95 default value
# quant =  1-alpha/k/(k-1)
# zi = ndtri(quant)
# crit = zi*np.sqrt(n*k*(k+1)/6) # always divide by 6
    
#     # create table
#     cols = ['DEM'] + dems_list
#     df_table = pd.DataFrame(columns=cols) # df and cols names
#     df_table['DEM'] = dems_list # first column of df

#     # get ranks values 
#     ranks_vals = dems_ranked.to_frame().T
dem_list  = list(df_criteria_full.columns)[crit_idx+1:]
first = second = dem_list
combined = [(f,s) for f in first for s in second if f!=s]

    # populate table
for pair in combined:
    d1 = pair[0]
    d2 = pair[1]
    print(f'{d1} x {d2} w = {w}')
    w, p = stats.wilcoxon(df_ranks[d1],df_ranks[d2],alternative='greater')
#     print(f'w: {w}')
#     print(f'p: {p}')
    
    
#             # print(d1,d2,rank_dem1,rank_dem2)
#             if np.abs(rank_dem1 - rank_dem2) > crit:
#                 df_table.at[r,d2] = 'Y'
#             else:
#                 df_table.at[r,d2] = 'N'

#     # use numpy to get only the upper triangle of the table 
#     m = np.triu(df_table.values,k=2)
#     df2 = pd.DataFrame(m,columns=cols)
#     df2['DEM'] = dems_list
#     # return df2
#     return df2


### 7 - Make some plots


In [ ]:
# main options for plots (colors, symbols etc)

# dem order
dem_order = ['ALOS','COP','FABDEM','NASA','SRTM','ASTER']
dem_order_rank = [i+'_rank' for i in dem_order]
# dem_colors_rank = {'ALOS_rank':'firebrick','COP_rank':'royalblue','FABDEM_rank':'forestgreen','NASA_rank':'darkorchid','SRTM_rank':'darkorange','ASTER_rank':'darkgoldenrod'}
# colors = ['firebrick','royalblue','forestgreen','darkorchid','darkorange','darkgoldenrod']

# symbols
symbols = ['X','o','s','^','v','d']

# http://scipy.github.io/old-wiki/pages/Cookbook/Matplotlib/Show_colormaps

In [ ]:
# create a dataframe for plotting the normalized values for each criterion
# Uses the unfiltered 'df_criteria' dataframe!
        
# create dict with each criterion in CRITERION column 
# and each type in REF_TYPE and LAND_TYPE columns
crit_dict = {target:'CRITERION' for target in df_criteria['CRITERION'].unique()}
type_dict = {
    'DSM':'REF_TYPE',
    'DTM':'REF_TYPE',
    'ALL':'LAND_TYPE',
    'FLAT':'LAND_TYPE',
    'STEEP':'LAND_TYPE',
    'URBAN':'LAND_TYPE',
    'FOREST':'LAND_TYPE'}
eq_dict ={**type_dict,**crit_dict}

# create dict to hold greater than conditions
gt_dict={
    'Urban > 25 %':'URBAN_PC',
    'Forest > 50 %':'FOREST_PC',
    'Avg Rough > 10 m':'AVG_ROUGH',
    'Relief > 500 m':'RELIEF',
    'Avg Slope > 18 %':'AVG_SLOPE'}

# create dict to hold less than conditions
lt_dict={
    'Avg Rough < 5 m':'AVG_ROUGH',
    'Relief < 25 m':'RELIEF',
    'Avg Slope < 18 %':'AVG_SLOPE'}

# list of labels to define order of row for plotting
list_yaxis = ['Forest > 50 %', 
'Urban > 25 %',
'Avg Rough < 5 m', 
'Avg Rough > 10 m', 
'Relief > 500 m',
'Relief < 25 m',
'Avg Slope > 18 %', 
'Avg Slope < 18 %', 
'DSM', 
'DTM', 
'FOREST', 
'URBAN', 
'STEEP',
'FLAT', 
'ALL', 
'ELD_LE90',
'ELD_MAE', 
'ELD_RMSE', 
'ELD_STD', 
'ELD_AVG', 
'SMD_LE90', 
'SMD_MAE', 
'SMD_RMSE', 
'SMD_STD', 
'SMD_AVG', 
'RUFD_LE90', 
'RUFD_MAE', 
'RUFD_RMSE', 
'RUFD_STD', 
'RUFD_AVG']

# get ranks for conditions and make dataframes
# ideally, this should be run with the full df_ranks dataframe 
df_eq_crit = dw.get_ranks_for_equal_criteria(df_ranks,eq_dict,dem_list)
df_gt_crit = dw.get_ranks_for_gt_criteria(df_ranks,gt_dict,dem_list)
df_lt_crit = dw.get_ranks_for_lt_criteria(df_ranks,lt_dict,dem_list)

# create dataframe for plotting 
df_plot = pd.concat([df_gt_crit,df_lt_crit,df_eq_crit])

# normalize values per row
df_plot_norm = df_plot.div(df_plot.max(axis=1), axis=0)

# df_plot_norm #uncomment to see the dataframe

In [ ]:
# define df and order of rows for plotting
df = df_plot_norm.loc[list_yaxis][::-1]

# plot
fig, ax = plt.subplots(figsize=(10,15))
for dem,s in zip(dem_list,symbols):
    ax.scatter(y=df.index, x=df[dem], label=dem, marker=s, s=50, cmap='Paired', alpha=0.8)

# decorations
ax.set_title('Normalized ranks', fontdict={'size':18})
ax.set_xlabel('Normalized ranks')
ax.set_yticks(df.index)
ax.legend()
plt.show()

In [ ]:
# create a dataframe for plotting the metrics values for selected tiles
        
# selected tiles
tiles_list= [
'N59TE010C',
'N59TE010B',
'N59RE010C',
'N59RE010B',
'N59RE0101',
'N59RE009G',
'N43PW002C',
'N43PW002B',
'N28XW018B',
'N28VW018B',]

# selected criteria
crit_list = [
'ELD_LE90']#,
#'ELD_RMSE']

# select by tile, by criteria, and only with 'LAND_TYPE'=='ALL'
df = df_ranks
df_select = df.loc[(df['DEMIX_TILE'].isin(tiles_list)) & (df['CRITERION'].isin(crit_list)) & (df['LAND_TYPE']=='ALL')]

# select only needed cols and set index 
df_select = df_select[['DEMIX_TILE','REF_TYPE','CRITERION'] + dem_order].set_index('DEMIX_TILE')


In [ ]:
# plot

# figure
fig, (ax1, ax2) = plt.subplots(2, sharex=True, figsize=(10,15))

# dsm
ax1.set_ylabel('DSM')
df = df_select.loc[df_select['REF_TYPE']=='DSM']
for tile,dem,s in zip(tiles_list,dem_list,symbols):
    ax1.scatter(y=df.index, x=df[dem], label=dem, marker=s, s=80, cmap='Paired', alpha=0.8)
ax1.legend()

# dtm
ax2.set_ylabel('DTM')
df = df_select.loc[df_select['REF_TYPE']=='DTM']
for tile,dem,s in zip(tiles_list,dem_list,symbols):
    ax2.scatter(y=df.index, x=df[dem], label=dem, marker=s, s=80, cmap='Paired', alpha=0.8)
    
# decorations
fig.suptitle('ELD_LE90', fontsize=14, y=0.9);
plt.subplots_adjust(hspace=0.05)


In [ ]:
# create a dataframe for plotting the counting of ranks for each DEM
# Uses the unfiltered 'df_criteria' dataframe!

ranks_cols = [i+'_rank' for i in dem_list]
df_only_ranks = df_ranks[ranks_cols]
df_counts = df_only_ranks.apply(pd.value_counts)

# df_counts[dem_order_rank] # uncomment to see the dataframe

In [ ]:
# plot stacked bar chart of ranks - by DEM
# color=[plt.cm.Paired(np.arange(len(df)))])

df = df_counts[dem_order_rank].T
ax = df.plot(kind='barh', stacked=True, figsize=(12,5), sort_columns=True, colormap='RdBu', alpha=0.6)
ax.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
ax.set_title('Ranks by DEM');

In [ ]:
# plot stacked bar chart of rank - by rank value
df = df_counts[dem_order_rank]
ax = df.plot(kind='barh', stacked=True, figsize=(12,8), colormap='Paired', alpha=0.8)#, color=dem_colors_rank)
ax.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
ax.set_title('Ranks by value');

In [ ]:
# create a dataframe for plotting the counting of ranks for each criterion

crit_list = ['ELD_LE90',
'ELD_MAE', 
'ELD_RMSE', 
'ELD_STD', 
'ELD_AVG', 
'SMD_LE90', 
'SMD_MAE', 
'SMD_RMSE', 
'SMD_STD', 
'SMD_AVG', 
'RUFD_LE90', 
'RUFD_MAE', 
'RUFD_RMSE', 
'RUFD_STD', 
'RUFD_AVG']

# subset ranks dataframe
dem_ranks = [i+'_rank' for i in dem_list]
ranks_cols = ['CRITERION']+dem_ranks
df_crit_ranks = df_ranks[ranks_cols]

# create new df
df_crit_low = pd.DataFrame(columns=ranks_cols)
df_crit_low['CRITERION'] = crit_list
# df_crit_low.set_index('CRITERION')

# populate df
for d in dem_ranks:
#     dfrow = dem_ranks.index(d)
    for c in crit_list:
        c_row = crit_list.index(c)
        try:
            rank_dem = df_crit_ranks[df_crit_ranks['CRITERION']==c][d].value_counts()[1.0]
        except:
            rank_dem = 0
        df_crit_low.at[c_row,d] = rank_dem

df_crit_low = df_crit_low.set_index('CRITERION')
        
# df_crit_low # uncomment to see the dataframe

In [ ]:
df = df_crit_low[dem_order_rank]
ax = df.plot(kind='barh', stacked=True, figsize=(12,8), colormap='Paired', alpha=0.8)
ax.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
ax.set_title('Number of times a DEM is best');